In [1]:
%reload_ext autoreload
%autoreload 2
%reload_ext notexbook
%texify

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import time
import sys
sys.setrecursionlimit(10000)
%matplotlib qt5

In [3]:
import cellbgnet
import cellbgnet.utils
from cellbgnet.datasets import DataSimulator
from cellbgnet.utils.hardware import cpu, gpu
from cellbgnet.model import CellBGModel

In [4]:
param_file = '../../cellbgnet/utils/reference_files/reference.yaml'
param = cellbgnet.utils.param_io.ParamHandling().load_params(param_file)

In [5]:
psf_params = param.PSF.to_dict()
simulation_params = param.Simulation.to_dict()
hardware_params = param.Hardware.to_dict()
eval_params = param.Evaluation.to_dict()
train_size = simulation_params['train_size']

In [6]:
model = cellbgnet.model.CellBGModel(param)

training sliding windows on camera chip:
Area num: 0, field_xy: [0, 127, 0, 127]
Area num: 1, field_xy: [114, 241, 0, 127]
Area num: 2, field_xy: [228, 355, 0, 127]
Area num: 3, field_xy: [342, 469, 0, 127]
Area num: 4, field_xy: [456, 583, 0, 127]
Area num: 5, field_xy: [570, 697, 0, 127]
Area num: 6, field_xy: [684, 811, 0, 127]
Area num: 7, field_xy: [798, 925, 0, 127]
Area num: 8, field_xy: [912, 1039, 0, 127]
Area num: 9, field_xy: [1026, 1153, 0, 127]
Area num: 10, field_xy: [1140, 1267, 0, 127]
Area num: 11, field_xy: [1174, 1301, 0, 127]
Area num: 12, field_xy: [0, 127, 114, 241]
Area num: 13, field_xy: [114, 241, 114, 241]
Area num: 14, field_xy: [228, 355, 114, 241]
Area num: 15, field_xy: [342, 469, 114, 241]
Area num: 16, field_xy: [456, 583, 114, 241]
Area num: 17, field_xy: [570, 697, 114, 241]
Area num: 18, field_xy: [684, 811, 114, 241]
Area num: 19, field_xy: [798, 925, 114, 241]
Area num: 20, field_xy: [912, 1039, 114, 241]
Area num: 21, field_xy: [1026, 1153, 114, 24

In [7]:
model.init_recorder()
model.init_eval_data()

Eval image generation: 100%|████████████████████████████████████████| 30/30 [00:13<00:00,  2.30it/s]


eval images shape: (30, 1041, 1302) contain 21582 molecules,


In [8]:
model.evaluation_params['eval_imgs'].shape

(30, 1041, 1302)

In [9]:
simulation_params

{'train_size': 128,
 'train_type': 'cells',
 'psf_intensity_normalization': True,
 'density': 0.00625,
 'non_cell_density': 10,
 'margin_empty': 0.05,
 'z_prior': [-1, 1],
 'survival_prob': None,
 'min_photon': 0.25,
 'camera': 'sCMOS',
 'camera_chip_size': [1041, 1302],
 'perlin_noise': False,
 'perlin_noise_factor': 0.2,
 'perlin_noise_res': 64,
 'baseline': 103.0,
 'bg_values': 112.0,
 'e_per_adu': 0.38,
 'qe': 0.95,
 'spurious_c': 0.0,
 'sig_read': '/mnt/sda1/SMLAT/data/real_data/dark_images/RN_e_venus_upgraded.tiff',
 'robust_training': False,
 'cell_masks_dir': '/mnt/sda1/SMLAT/data/real_data/chromosome_dots/EXP-23-CA3045/pool_3045_ter_pos01/phase_venus_mask/',
 'cell_masks_filetype': '.png',
 'edt_noise_map_path': '/mnt/sda1/SMLAT/data/real_data/chromosome_dots/EXP-23-CA3045/edt_noise.pkl',
 'min_edt': 0,
 'max_edt': 6,
 'augment': True}

In [10]:
start_time = time.time()
model.fit(batch_size=64, max_iters=40000, print_output=True, print_freq=100)
duration = time.time() - start_time

Started training .... 
 || Cost: 2177.807 || Time Upd.: 900.9 ms  || BatchNr.: 100
 || Cost: 173.523 || Time Upd.: 872.0 ms  || BatchNr.: 200
 || Cost: 111.184 || Time Upd.: 876.6 ms  || BatchNr.: 300
 || Cost: 91.577 || Time Upd.: 873.9 ms  || BatchNr.: 400
 || Cost: 75.743 || Time Upd.: 914.6 ms  || BatchNr.: 500
 || Cost: 70.231 || Time Upd.: 890.5 ms  || BatchNr.: 600
 || Cost: 66.727 || Time Upd.: 897.9 ms  || BatchNr.: 700
 || Cost: 67.273 || Time Upd.: 896.4 ms  || BatchNr.: 800
 || Cost: 60.980 || Time Upd.: 885.9 ms  || BatchNr.: 900
 || Cost: 53.855 || Time Upd.: 883.0 ms  || BatchNr.: 1000
processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 21582FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 21539 ,preds: 17184
JoR: 1.590 || Eff_3d: 44.033 || Jaccard: 0.794 || Factor: 838.434 || RMSE_lat: 49.962 || RMSE_ax: 108.251 || RMSE_x: 35.2

processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 21582FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 21539 ,preds: 16805
JoR: 2.011 || Eff_3d: 54.748 || Jaccard: 0.779 || Factor: 596.347 || RMSE_lat: 38.738 || RMSE_ax: 80.466 || RMSE_x: 28.253 || RMSE_y: 26.502 || Cost: 42.108 || Recall: 0.780 || Precision: 0.999 || BatchNr.: 2200 || Time Upd.: 880.2 ms 
 || Count Cost: 28.7712 || Localization Cost: 24.4178 || Bg Cost: 0.0554 || P_locs Cost: 30.0018saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 54.748 || 38.738 || 80.466 || 2200


processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 21582FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 21539 ,preds: 18268
JoR: 1.943 || Eff_3d: 54.2

processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 21582FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 21539 ,preds: 17290
JoR: 2.092 || Eff_3d: 56.403 || Jaccard: 0.801 || Factor: 566.051 || RMSE_lat: 38.316 || RMSE_ax: 78.622 || RMSE_x: 27.603 || RMSE_y: 26.574 || Cost: 38.919 || Recall: 0.802 || Precision: 0.999 || BatchNr.: 3400 || Time Upd.: 883.3 ms 
 || Count Cost: 29.3852 || Localization Cost: 21.9321 || Bg Cost: 0.0470 || P_locs Cost: 29.3950saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 56.403 || 38.316 || 78.622 || 3400


processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 21582FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 21539 ,preds: 18239
JoR: 2.040 || Eff_3d: 53.3

processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 21582FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 21539 ,preds: 19093
JoR: 2.170 || Eff_3d: 58.218 || Jaccard: 0.882 || Factor: 716.518 || RMSE_lat: 40.660 || RMSE_ax: 79.041 || RMSE_x: 29.189 || RMSE_y: 28.306 || Cost: 38.681 || Recall: 0.884 || Precision: 0.998 || BatchNr.: 4600 || Time Upd.: 888.9 ms 
 || Count Cost: 28.1504 || Localization Cost: 21.9860 || Bg Cost: 0.0475 || P_locs Cost: 29.0538saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 58.218 || 40.660 || 79.041 || 4600


processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 21582FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 21539 ,preds: 18984
JoR: 2.160 || Eff_3d: 57.3

processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 21582FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 21539 ,preds: 19009
JoR: 2.212 || Eff_3d: 58.545 || Jaccard: 0.880 || Factor: 707.471 || RMSE_lat: 39.786 || RMSE_ax: 79.149 || RMSE_x: 28.502 || RMSE_y: 27.759 || Cost: 35.250 || Recall: 0.881 || Precision: 0.998 || BatchNr.: 5800 || Time Upd.: 879.7 ms 
 || Count Cost: 27.9281 || Localization Cost: 19.4326 || Bg Cost: 0.0396 || P_locs Cost: 28.2544saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 58.545 || 39.786 || 79.149 || 5800


processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 21582FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 21539 ,preds: 18592
JoR: 2.148 || Eff_3d: 58.2

...

processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 21582FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 21539 ,preds: 19601
JoR: 2.365 || Eff_3d: 60.952 || Jaccard: 0.906 || Factor: 690.659 || RMSE_lat: 38.319 || RMSE_ax: 75.000 || RMSE_x: 27.296 || RMSE_y: 26.894 || Cost: 25.346 || Recall: 0.908 || Precision: 0.998 || BatchNr.: 34600 || Time Upd.: 869.5 ms 
 || Count Cost: 26.8938 || Localization Cost: 11.4550 || Bg Cost: 0.0237 || P_locs Cost: 26.4661saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 60.952 || 38.319 || 75.000 || 34600


processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 21582FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 21539 ,preds: 19674
JoR: 2.365 || Eff_3d: 61

processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 21582FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 21539 ,preds: 19431
JoR: 2.395 || Eff_3d: 62.016 || Jaccard: 0.899 || Factor: 712.519 || RMSE_lat: 37.554 || RMSE_ax: 71.399 || RMSE_x: 26.833 || RMSE_y: 26.273 || Cost: 25.545 || Recall: 0.901 || Precision: 0.998 || BatchNr.: 35800 || Time Upd.: 874.8 ms 
 || Count Cost: 26.9987 || Localization Cost: 11.5942 || Bg Cost: 0.0241 || P_locs Cost: 26.5018saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 62.016 || 37.554 || 71.399 || 35800


processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 21582FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 21539 ,preds: 19424
JoR: 2.391 || Eff_3d: 61

processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 21582FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 21539 ,preds: 19707
JoR: 2.376 || Eff_3d: 60.162 || Jaccard: 0.911 || Factor: 709.455 || RMSE_lat: 38.333 || RMSE_ax: 78.646 || RMSE_x: 27.407 || RMSE_y: 26.801 || Cost: 25.864 || Recall: 0.913 || Precision: 0.998 || BatchNr.: 37000 || Time Upd.: 891.6 ms 
 || Count Cost: 27.0020 || Localization Cost: 11.8730 || Bg Cost: 0.0244 || P_locs Cost: 26.5651saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 60.162 || 38.333 || 78.646 || 37000


processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 21582FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 21539 ,preds: 19811
JoR: 2.390 || Eff_3d: 60

processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 21582FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 21539 ,preds: 19501
JoR: 2.325 || Eff_3d: 59.281 || Jaccard: 0.902 || Factor: 699.685 || RMSE_lat: 38.795 || RMSE_ax: 80.491 || RMSE_x: 27.752 || RMSE_y: 27.109 || Cost: 25.066 || Recall: 0.904 || Precision: 0.998 || BatchNr.: 38200 || Time Upd.: 870.2 ms 
 || Count Cost: 26.7784 || Localization Cost: 11.2326 || Bg Cost: 0.0235 || P_locs Cost: 26.3997saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 59.281 || 38.795 || 80.491 || 38200


processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 21582FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 21539 ,preds: 19651
JoR: 2.388 || Eff_3d: 62

processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 21582FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 21539 ,preds: 19603
JoR: 2.381 || Eff_3d: 60.946 || Jaccard: 0.907 || Factor: 692.625 || RMSE_lat: 38.097 || RMSE_ax: 75.531 || RMSE_x: 27.273 || RMSE_y: 26.601 || Cost: 25.584 || Recall: 0.908 || Precision: 0.998 || BatchNr.: 39400 || Time Upd.: 873.8 ms 
 || Count Cost: 27.0536 || Localization Cost: 11.5994 || Bg Cost: 0.0241 || P_locs Cost: 26.5882saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 60.946 || 38.097 || 75.531 || 39400


processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 21582FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 21539 ,preds: 19690
JoR: 2.388 || Eff_3d: 61

In [11]:
from cellbgnet.utils.plot_funcs import plot_train_record

In [12]:
plot_train_record(model)